In [53]:
import sentencepiece as spm
from fastai.text import *
from torch.utils.data.dataset import Dataset


In [54]:
WGTS_PATH = './v50knl4/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5'
SP_MODEL_PATH = './v50knl4/work/up_low50k/tmp/sp-50k.model'

In [56]:
sp = spm.SentencePieceProcessor()
sp.load(SP_MODEL_PATH)

sp.SetEncodeExtraOptions("bos:eos")
sp.SetDecodeExtraOptions("bos:eos")

True

In [62]:
bptt=5
vs = len(sp)
em_sz,nh,nl = 400 ,1150,4
max_seq = 1000000
pad_token = 1
bidir = False
qrnn = False

In [66]:
rnn = MultiBatchRNN(bptt, max_seq, vs, em_sz, nh, nl, pad_token,
                    bidir, qrnn)

model = SequentialRNN(rnn, LinearDecoder(vs, em_sz, 0, tie_encoder=rnn.encoder))

load_model(model[0], WGTS_PATH)
model.reset()

In [67]:
model.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [68]:
class TextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

In [133]:
def predict(sentence): 
    ids = [np.array(sp.encode_as_ids(sentence))]

    dataset = TextDataset(ids)
    sampler = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(dataset, batch_size=100, transpose=True, pad_idx=1, sampler=sampler, pre_pad=False)

    tensors = None
    with no_grad_context():
        for (x, y) in dl:
            tensors, _, _ = model(x)
            
    last_tensor = tensors[-1]

    best = int(torch.multinomial(tensors[-1].exp(), 1))
    word = sp.decode_ids([best])
    
    while best in ids[0] or not word.isalpha():
        last_tensor[best] = -1
        best = int(torch.argmax(last_tensor))
        word = sp.decode_ids([best])

    return word

In [283]:
def predict_sentence(sentence, length):
    result = sentence
    res_list = []
    
    for i in range(length):
        word = predict(result)
        if len(word) > 0 and word[0].isalpha() and (len(word) > 2 or word.lower() in ['a', 'o', 'i', 'w', 'z']):
            result += (" " + word)
            res_list.append(word)
        else:
            result += word
            res_list.append(word)
    return result, res_list

In [156]:
r, rl = predict_sentence("Warszawa to największe", 25)

In [159]:
print(f"(M){r}")

(M)Warszawa to największe miasto na znajdującym się w pobliżu centrum miasta i Hawan a także Chin a konkretnie Marsa oraz Burund i Glasgow z prawosławnych miast o


In [162]:
r, rl = predict_sentence("Te zabawki należą do", 15)

In [164]:
print(f"(D){r}")

(D)Te zabawki należą do bardzo podobnych w tym temacie i zachęcają dzieci na bezpłatną wycieczkę po całym


In [184]:
r, rl = predict_sentence("Policjant przygląda się", 15)

In [185]:
print(f"(C){r}")

'Policjant przygląda się prze kazaniu hipnozy adapt ującego trolejbusow ron dzie sl źc w to Pensjonat Monroe'

In [191]:
r, rl = predict_sentence("Na środku skrzyżowania widać", 15)

In [192]:
print(f"(B){r}")

'Na środku skrzyżowania widać ślady planowania pracy zobowiąza lności za umiejętność klasyfikowani a refleksyjn ość daje równe szanse'

In [203]:
r, rl = predict_sentence("Właściciel samochodu widział złodzieja z", 15)

In [205]:
print(f"(N){r}")

(N)Właściciel samochodu widział złodzieja z ograniczonym i przetwórcą wymagane identyfikatory ją Paweł używając Res ety ka średnio on


In [206]:
r, rl = predict_sentence("Prezydent z premierem rozmawiali wczoraj o", 15)

In [207]:
print(f"(Ms){r}")

(Ms)Prezydent z premierem rozmawiali wczoraj o potrzebie remontu zaplecza techniczn ego obejmującym szpitale kliniczne i kasy chorych NFZ w ramach konkursu


In [252]:
r, rl = predict_sentence("Witaj drogi", 15)

In [253]:
print(f"(W){r}")

(W)Witaj drogi Cesarz zadawał sut o ctl wyzwaniu i paragraf więzi eń przesłany przez


In [254]:
gender_sentence_male = "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie"
gender_sentence_female = "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie"

In [258]:
r, rl = predict_sentence(gender_sentence_male, 25)

In [260]:
print(f"(MALE) {r}")

(MALE) Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie wypłacił a ja podejrzewam w ostateczności jaki zadziałał termin na otwieranie tego przedłużenie i jak dodał em proto nadania z niego do stając ego


In [269]:
r, rl = predict_sentence(gender_sentence_female, 25)

In [270]:
print(f"(FEMALE) {r}")

(FEMALE) Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie stres owała plus wąwoz wyeksportowa ła giew od ego Lo ody top work i zielonego ot uchy li che de la gol czworobok darmowe


In [278]:
long = "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"

In [284]:
r, rl = predict_sentence(long, 50)

In [285]:
r

'Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane brazylijskim psem biodrowym z plemieni a pole golfowe swepy pieniedz a zwołac sklad ających konido posaż a tylko tujemy opuszczani petentie nauczycielski ego nosc Podkarpacia Trening ową Odzież Boże Groźn czkądoch a seans zdobywani a wybici a nie lekarstw o dejmowanie'